<a href="https://colab.research.google.com/github/breaks-software/Candida-CRISPR-Target-Notebook/blob/master/Candida_CRISPR_Cas9_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Candida CRISPR-Cas9 Workbook

Set up required packages:

In [ ]:
#@title Press the Run button to set up all of the required dependencies
from urllib import request
from gzip import open as gzip_open

# use biopython package to work with sequences
!pip3 install biopython
from Bio import SeqIO

# install chromedriver and selenium so that we can work with the EuPaGDT site
!apt-get update
!apt install chromium-chromedriver
!pip3 install selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# install the pandas package to be able to easily manipulate the data
!pip3 install pandas
from pandas import DataFrame, concat, read_html, options

# and we need a tool to display our results in the notebook
from IPython.display import display

# we need to be able to download the results
from google.colab import files

---
## Collect the gene sequence
In this section, we'll collect the full sequence for our target gene.

In [ ]:
#@title Specify your target gene below, then press run:

target_gene_id = "CPAR2_404800" #@param {type:"string"}

In [ ]:
#@title Run to download the sequence file for C. parapsilosis from the Candida Genome Database, uncompress the file for processing, then collect the sequence for our target gene
candida_genome_database_url = 'http://www.candidagenome.org/download/sequence/C_parapsilosis_CDC317/current/C_parapsilosis_CDC317_current_orf_genomic.fasta.gz'

with request.urlopen(candida_genome_database_url) as response:
    with gzip_open(response, mode="rt") as uncompressed:
        input_seq_iterator = SeqIO.parse(uncompressed, "fasta")
        try:
          target_gene_sequence = [record for record in input_seq_iterator \
                                  if record.id == target_gene_id][0]
        except IndexError:
          raise IndexError("there is no sequence record for id = {}".format(target_gene_id))      

In [ ]:
#@title Run to review the gene sequence we found matching the specified target id:
#@markdown Hints, 
#@markdown * the "Description" should be reasonable,
#@markdown * and the [type of sequence](https://biopython.org/DIST/docs/api/Bio.Alphabet-module.html) specified at the end (e.g. SingleLetterAlphabet()) should match the type of sequence you expected.]
print(target_gene_sequence)

# Identify target insertion sites:
The preference here is to insert guide RNA as early in the sequence as possible, so we want to select a target portion of the sequence.  If the sequence is long enough, target the first 2000 bases (1kb).  If the sequence is shorter than that, take the first half.  If the sequence is really short, **manually override** the sub_sequence_start and sub_sequence_length variable values below.

In [ ]:
#@title Change the default start position (counting starts at 0) and length for the target subsequence: 
sub_sequence_start =  0  #@param {type: "integer"}
sub_sequence_length = 4017  #@param {type: "integer"}
sequence_length = len(str(target_gene_sequence.seq))
#@markdown ###  or insert your custom subsequence here (be sure to clear this if you don't want to set a custom subsequence):
custom_sub_sequence = 'ATGGTCAAACACTTGCAATTTGTGGCGATATTGGTGGCCTTTACACTTACAGCCCTCACTCAAGCAGCTGAGATTAGCAATGTTTTCCAAAGCTTTGATAGTTTAACTTGGGAAAACGGTGCAAGCTATAGGTACAGAACCCCTTTAACTCCTAGTTGGATTGCACAATTGTCTTGGAAGATTTTAGGTTCCAATGTCAAACCAGGCGACACATTCACATTGAACATGCCATGTGTATTCAAGTTTACAACAACACAAGAGAGTATTGACTTGGACGTCGGAGACACGGTATATGCTACATGTCGTTTCGAACCAGGTGATTTAGTCGTTGCATATTCACAATTGAAATGTACTGCCAGCGATAATGTTAAAGATAGCACTGATGCTACTGGTACTGTTAGATTCCCTTTCACATTCAAtagatagatagtggccgcatttcgcagatgtTGTTGGTGGGTCTGCTGGTGTAGTTGATTTGCAAAACTCAAAATGTTTCACACCAGGTACTAATGAAGT' #@param {type: "string"}

if sequence_length < sub_sequence_length:
  sequence_length = sequence_length // 2

if len(custom_sub_sequence) != 0:
  subsequence = custom_sub_sequence
else:
  subsequence = str(target_gene_sequence.seq)[sub_sequence_start : sub_sequence_start + sub_sequence_length]

from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

print("Subsequence length:  {}".format(len(subsequence)))
print("Subsequence: \n{}".format(subsequence))

We'll use the [Eukaryotic Pathogen CRISPR guide RNA/DNA Design Tool](http://grna.ctegd.uga.edu/) to identify potential oligos for our CRISPR target.

You need to provide a "Job Name" to EuPaGDT (the default name is the CPAR2 id followed by the size of the subsequence):

In [ ]:
#@title Specify your EuPaGDT job name:
job_name = "Als7-repair" #@param {type:"string"}

The next step is to prepare values for the different fields that we have to submit when we request guide RNA:

In [ ]:
#@title Fill in the relevant EuPaGDT form fields:
rna_guided_nuclease_selection = "SpCas9"  #@param ["SpCas9"] {type: "raw"} #
# unsupported options: "SaCas9", "AsCpf1", "CjCas9", "CjCas9_2", "FnCas9", "FnCpf1", "LbCpf1", "NmCas9", "St1Cas9", "TdCas9"]
gRNAlength = 20  #@param {type:"integer"}
PAM = "NGG"  #@param {type:"string"}
homology_arm_length = 30  #@param {type:"integer"}
HDR_repair_template_insertion = ""  #@param {type: "string"}
genome = "C_parapsilosis_CDC317_current_chromosomes"  #@param {type:"string"}
three_pam_pos = True
# the subsequence to search was determined in a previous step

form_fields = {
    "name": (None, job_name),
    "proteinselection": (None, rna_guided_nuclease_selection),
    "MH_switch": (None, 'off'),  # Microhomology search
    "conreg_switch": (None, 'off'),  # Conserved region search
    "textbox1": (None, subsequence),
    "gRNAlength": ("ui-id-27", gRNAlength),
    "PAM": ("ui-id-27", PAM),
    "PAMpos": ("ui-id-27", "3PAM" if three_pam_pos else "5PAM"),
    "onTflanklen": ("ui-id-29", "50"),
    "onTidentity": ("ui-id-29", "0.7"),
    "onTcoverage": ("ui-id-29", "0.7"),
    "seedlength": ("ui-id-31", "15"),
    "maxnummismatch": ("ui-id-31", "3"),
    "offPAM": ("ui-id-31", "NAG,NGA"),
    "HDRflank": ("ui-id-33", homology_arm_length),
    "HDRinsertion": ("ui-id-33", HDR_repair_template_insertion),
    "genome": ("ui-id-5", genome),
}


In [ ]:
#@title Click the run icon to submit your request to find gRNA sequences from EuPaGDT
# because the EuPaGDT site uses CGI scripts that are not open source,
# and there is no evidence that it provides an API on its server,
# we will have to emulate using a browser to make our requests.
# using the Selenium tool to do this.

# first, create some automation code that knows how to interact with the EuPaGDT page
class EuPaGDTPage(object):
  def __init__(self, driver, form_fields):
    """
    takes a selenium webdriver instance, and a dictionary specifying the 
    field names and desired values in the page's form
    """
    self.driver = driver
    self.main_tab_handle = self.driver.window_handles[0]
    self.form_fields = form_fields

  def submit_request(self):
      """ submit the request """
      for field_name in self.form_fields:
          accordion_id, field_value = form_fields[field_name]
          if accordion_id is not None:
              accordion = self.driver.find_element_by_id(accordion_id)
              controlled_div_id = accordion.get_attribute("aria-controls")
              controlled_div = self.driver.find_element_by_id(controlled_div_id)
              if not controlled_div.is_displayed():
                  accordion.click()

          # Now try to find the field...
          try:
              field = self.driver.find_element_by_id(field_name)
          except NoSuchElementException:
              # some do not have the id attribute, so try looking by name
              field = self.driver.find_element_by_name(field_name)

          # once we've expanded enough accordions, we have to be sure to make the target field visible in the window
          self.driver.execute_script("arguments[0].scrollIntoView();", field)
          if field.tag_name == "select":
              Select(field).select_by_value(field_value)
          elif field.tag_name == "input":
              if field.get_attribute("type") == "radio":
                  radio_option = self.driver.find_element_by_css_selector(
                      "input[name='{}'][value='{}']".format(field_name, field_value))
                  self.driver.execute_script("arguments[0].scrollIntoView();", radio_option)
                  radio_option.click()
              else:
                  field.send_keys(field_value)
          elif field.tag_name == "textarea":
              field.send_keys(field_value)

      form = self.driver.find_element_by_name("nameemailseq")
      form.submit()

  def get_grna_table(self):
    """ retrieve the html table with the results. """
    for window_handle in self.driver.window_handles:
      if window_handle != self.main_tab_handle:
        self.driver.switch_to.window(window_handle)

    # confirm that we're in the correct tab for the run status:
    if "Design Tool" in self.driver.title:
      raise Exception("could not find the gRNA finder browser tab")
    report_handle = self.driver.current_window_handle

    # the cgi is frequently reloading this page, which may result in a race condition where our target_links
    # become stale.
    # go find the target link...the one with the right text, but not the one for "conserved"
    grna_finder_timeout = 4*60
    target_link = [a for a in WebDriverWait(self.driver, grna_finder_timeout).until(
      EC.presence_of_all_elements_located((By.LINK_TEXT, "gRNA sequence and score")),
      "gRNA finder browser page did not complete in {} seconds".format(grna_finder_timeout))
    if "conserved" not in a.get_attribute("href")][0]

    target_link.click()

    # the tool is generating yet another tab
    for window_handle in self.driver.window_handles:
      if window_handle not in (self.main_tab_handle, report_handle):
        self.driver.switch_to.window(window_handle)

    # find the HTML table containing the results of the search
    table = WebDriverWait(self.driver, 60).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, "table")))
    # load it into a pandas dataframe for easier manipulation
    data_frames = read_html(table.get_attribute("outerHTML"), header=0)
    df = data_frames[0]

    # We are only interested in entries that have no off-target hits
    # and on-target hits that are conditional on if we 
    # specified a custom_sub_sequence
    if len(custom_sub_sequence) == 0:
      on_target_hits = '1 | 0'
    else:
      on_target_hits = '0 | 0'
    
    f = df[(df['On-target hits in the genome (perfect-match | non-perfect-but-PAM-match)']==on_target_hits) & (df['Off-target hits (perfect-match | nonperfect-match)']=='0 | 0')]

    # we've observed that we're still seeing duplicate entries of gRNA sequences, even with single on-target hits
    # so we'll find any that are duplicated and ignore those
    dupped = f.duplicated(subset=['gRNA sequence (PAM "NGG" )'], keep=False)
    # report any that are not duplicated and sort tyem by the gRNA sequence
    no_dups = f[~dupped]

    # we need to remove any rows that have the potential problem of 4 or more T(U)s in a row
    contains_4_or_more = no_dups['Potential problems during transcription'].str.contains('gRNA contains >=4')
    no_or_limited_potential_problems = no_dups[~contains_4_or_more]

    # now, we need to expand our set for any that require an A or a G prefix
    rows_require_g_or_a = no_or_limited_potential_problems[
      no_or_limited_potential_problems['Potential problems during transcription'].str.contains('please manually add a leading')]

    # Do this by duplicating this set of rows twice, providing one of the prefixes to each set
    rows_with_g_prefix = rows_require_g_or_a.copy()
    rows_with_g_prefix['gRNA sequence (PAM "NGG" )'] = 'G' + rows_with_g_prefix['gRNA sequence (PAM "NGG" )']
    rows_with_a_prefix = rows_require_g_or_a.copy()
    rows_with_a_prefix['gRNA sequence (PAM "NGG" )'] = 'A' + rows_with_a_prefix['gRNA sequence (PAM "NGG" )']

    # then removing the original rows and adding back the two new sets
    no_problems = no_dups[no_dups['Potential problems during transcription'] == 'No problem found']
    final_guide_rnas = no_problems.append(rows_with_g_prefix).append(rows_with_a_prefix).set_index('gRNA sequence (PAM "NGG" )')

    return final_guide_rnas.sort_values(by=['gRNA id'])

# set selenium webdriver options to be headless (so it won't bring up a browser)
# assumes that the Chrome browser is installed.
webdriver_options = webdriver.ChromeOptions()
webdriver_options.add_argument('--headless')
webdriver_options.add_argument('--no-sandbox')
webdriver_options.add_argument('--disable-dev-shm-usage')

# open the website, fill in the form, and wait for results
driver = webdriver.Chrome('chromedriver',
                          options=webdriver_options,
                          desired_capabilities=None,
                          service_log_path=None,
                          chrome_options=None,
                          keep_alive=True)
driver.get("http://grna.ctegd.uga.edu/")

eupagdt = EuPaGDTPage(driver, form_fields)
eupagdt.submit_request()
eupagdt_results = eupagdt.get_grna_table()
print("found {} gRNA sequences to explore".format(eupagdt_results.shape[0]))


# Verify target insertion sites against RNA Fold Server

In [ ]:
#@title Click the run icon to pull RNA Fold information for the gRNA sequences
# now update the table with information from the RNA Fold server about each guide RNA
# first, create some code to define how to interact with the RNA fold server

# Page object for the RNA Fold Server page
class RNAFoldServerPage(object):
  def __init__(self, driver):
    """
    Takes an instance of a selenium webdriver
    """
    self.driver = driver
    # keep track of the browser tab where we start from
    self.main_tab_handle = self.driver.window_handles[0]

  def retrieve_fold_results(self, guide_rna_sequence):
    """
    this method will populate the RNA Fold Server step 1 form
    and request that our guide_rna_sequence be sent for processing,
    then wait for the results before retrieving them.

    ..note::
    We are currently accepting all default form parameters, except for
    the sequence that we are asking to be screened.

    :param guide_rna_sequence: the gRNA sequence we need to check
    """
    form_fields = {
        "SCREEN": guide_rna_sequence,
        # "CONSTRAINT": "",
        # "FILE": "",
        "method": "p",
        "nocloseGU": False,
        "noLP": True,
        # "dangling": "nd2",
        # "param": "rna2004",
        # "SHAPEDATA": "",
        # "SHAPEFILE": "",
        # "shapemethod": "deigan",
        # "shape_slope": "1.9",
        # "shape_intercept": "-0.7",
        # "shape_beta": "0.8",
        # "deigan_conversion": "linearlog",
        # "shape_conv_cutoff": "0.25",
        # "shape_conv_linear_s": "0.68",
        # "shape_conv_linear_i": "0.2",
        # "shape_conv_linearlog_s": "1.6",
        # "shape_conv_linearlog_i": "-2.29",
        # "Temp": "37",
        "svg": True,
        "reliability": True,
        "mountain": True,
        # "EMAIL": "",
    }

    for field_name in form_fields:
      # Now try to find the field...
      try:
        field = self.driver.find_element_by_id(field_name)
      except NoSuchElementException:
        # some do not have the id attribute, so try looking by name
        field = self.driver.find_element_by_name(field_name)

      # and set our desired value
      self.driver.execute_script("arguments[0].scrollIntoView();", field)
      if field.tag_name == "select":
        Select(field).select_by_value(form_fields[field_name])
      elif field.tag_name == "input":
        if field.get_attribute("type") == "radio":
          radio_option = self.driver.find_element_by_css_selector(
            "input[name='{}'][value='{}']".format(field_name, form_fields[field_name]))
          self.driver.execute_script("arguments[0].scrollIntoView();", radio_option)
          radio_option.click()
        else:
          field.send_keys(form_fields[field_name])
      elif field.tag_name == "textarea":
        field.send_keys(form_fields[field_name])

    # submit the form for processing
    form = self.driver.find_element_by_name("form")
    form.submit()

    # wait for the page to refresh with our results...
    # we'll use a simple way to determine that, look for a link that will only show once completed.
    rna_fold_timeout = 4*60
    WebDriverWait(self.driver, rna_fold_timeout).until(
      EC.presence_of_all_elements_located((By.LINK_TEXT, "color by base-pairing probability")),
      "RNA fold server search request did not complete in {} seconds".format(rna_fold_timeout))

    # now go for the minimum free energy (MFE) numbers and the Centroid secondary structure svg diagram
    # the mfe number in kcal/mol is at the following css path
    mfe = self.driver.find_element_by_css_selector("p:nth-of-type(1) b").text

    # svg centroid secondary structure is inside an iframe look for the src containing "centroid.svg":
    # <iframe src="http://rna.tbi.univie.ac.at/RNAfold/wjqi1FDuxB/test_sequenc_centroid.svg" style="border:0;" width="452" height="650">
    iframes = self.driver.find_elements_by_tag_name('iframe')
    centroid_iframe = [f for f in iframes if 'centroid.svg' in f.get_attribute('src')][0]

    # return the MFE, and the link to the svg file displayed in the results
    return mfe, centroid_iframe.get_attribute("src")

# create a new pandas DataFrame with as many rows as we have gRNAs, and columns
# for the Minimum Free Energy and Centroid secondary structure SVG diagram
rna_fold_mfe_svgs_df = DataFrame(index=eupagdt_results.index, columns=['MFE', 'Centroid secondary structure'])

# parse through the guide RNAs to populate our new DataFrame
n = 0
for i in eupagdt_results.index:
    # strip off the PAM [last three characters] when we pass in our guide RNA
    driver.get("http://rna.tbi.univie.ac.at/cgi-bin/RNAWebSuite/RNAfold.cgi")
    rna_fold_server = RNAFoldServerPage(driver)

    n += 1
    s = i[:-3]
    print("\rretrieving {} of {} RNA Fold results for {}".format(n, rna_fold_mfe_svgs_df.shape[0], s), end="")

    mfe, centroid_svg = rna_fold_server.retrieve_fold_results(s)

    # add the results to our DataFrame
    rna_fold_mfe_svgs_df.at[i, 'MFE'] = mfe
    rna_fold_mfe_svgs_df.at[i, 'Centroid secondary structure'] = centroid_svg

# merge the MFE and associated SVG diagram with our guide RNA results
guide_rna_fold_mfe_svgs_df = concat([eupagdt_results, rna_fold_mfe_svgs_df], axis=1)



In [ ]:
#@title Display gRNA sequence scores with Minimum Free Energy and Centroid secondary structure
display_filters = False  #@param {type: "boolean"}

# present the data
options.display.max_columns = None
options.display.max_rows = guide_rna_fold_mfe_svgs_df.shape[0] + 1
displayed_columns = ["gRNA id", "Total score", "GC content", "efficiency score based on Doench et al.2014", "efficiency score based on CRISPRater", "MFE", "Centroid secondary structure"]
sort_by_columns = ['Total score', 'MFE']
sort_order = [False, True]

if display_filters:
  # use google.colab.data_table to display the DataFrame with search/sort tools
  %load_ext google.colab.data_table
  display(guide_rna_fold_mfe_svgs_df[displayed_columns].sort_values(sort_by_columns, ascending=sort_order))
else:
  # display the DataFrame using standard HTML of the Colab notebook
  # if we previously loaded the google.colab.data_table, unload it to avoid
  # a possible error later
  import subprocess
  output = subprocess.getoutput('jupyter nbextension list')
  if 'google.colab.data_table' in output:
    %unload_ext google.colab.data_table

  # provide some special formatting for the HTML results
  def path_to_image_html(path):
    # actually display the image that is referenced in the "Centroid..." column
    return '<img src="'+ path + '" width="60">'

  def highlight_gRNA(sequence):
    # we want to highlight the PAM part of the gRNA sequence
    pam_sequence = sequence[:-3] + '<span style="color: red;white-space: nowrap;">{}</span>'.format(sequence[-3:])
    final_sequence = pam_sequence
    # we want to highlight the leading A or G if it was prefixed
    if len(sequence) == gRNAlength + len(PAM) + 1:
      final_sequence = '<span style="color: green;">{}</span>'.format(pam_sequence[0]) + pam_sequence[1:]
    return final_sequence

  def highlight_mfe_0(mfe):
    # highlight if the MFE is zero
    if str(mfe) == "0.00":
      mfe = '<span style="color: red">{}</span>'.format(mfe)
    return mfe

  results_formatters = {
      "Centroid secondary structure": path_to_image_html, 
      '__index__': highlight_gRNA,
      "MFE": highlight_mfe_0
      }

  display(HTML(guide_rna_fold_mfe_svgs_df[displayed_columns].sort_values(sort_by_columns, ascending=sort_order).to_html(escape=False, formatters=results_formatters)))


In [ ]:
#@title Click Run to download the results table
results_file_name = "{}_gRNA_targets".format(job_name)
guide_rna_fold_mfe_svgs_df[displayed_columns].sort_values(sort_by_columns, ascending=sort_order).to_html(results_file_name + '.html', escape=False, formatters={"Centroid secondary structure": path_to_image_html})
files.download(results_file_name + '.html')
guide_rna_fold_mfe_svgs_df.sort_values(sort_by_columns, ascending=sort_order).to_csv(results_file_name + '.csv')
files.download(results_file_name + '.csv')

display("Look in your default Downloads folder for both a CSV and a HTML file with names of the pattern {}.*".format(results_file_name))
